In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="{subject}와 관련된 한국의 유명{subject}를 지역과 함께 5가지 리스트로 알려줘.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
pprint(prompt.partial_variables)

{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}


In [ ]:

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

chain = prompt | model | output_parser

result = chain.invoke({"subject": "음식"})

print("음식 목록:")
print(result)


음식 목록:
['김치(전라도', '서울)', '비빔밥(전북)', '불고기(경북)', '삼계탕(경남)', '냉면(강원)']


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 보통
input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 보통'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요\n2. 다른 설명이나 부가 설명을 추가하지 마세요\n3. 이모지나 특수문자도 포함하지 마세요\n4. 오직 하나의 단어만 출력하세요\n\n답변:'), additional_kwargs={})]


In [ ]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  
)

fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 3개 준비 완료


In [ ]:
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else enumParser)
        
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

=== 결과 요약 ===
성공: 3/3 (100.0%)
실패: 0/3


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

class SelfIntroduce(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="목표")
    
parser = PydanticOutputParser(pydantic_object=SelfIntroduce)

template = """
다음 사용자 에 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 먹표를 구조화된 형태로 추출해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "이름", "title": "Name", "type": "string"}, "age": {"description": "나이", "title": "Age", "type": "integer"}, "major": {"description": "전공", "title": "Major", "type": "string"}, "hobbies": {"description": "취미", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "목표", "title": "Goal", "type": "string"}}, "required": ["name", "age", "major", "hobbies", "goal"]}\n```'} messages=[Hu

In [ ]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"query": query})


print(output)

name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'


In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="언급된 여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="1~5점 사이의 추천점수"),
    ResponseSchema(name="activities", description="언급된 주요 활동을 리스트형태로 구조화한 형태")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

template = """
다음 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 언급된 여행지
	"duration": string  // 여행 기간
	"budget": string  // 여행 예산
	"rating": string  // 1~5점 사이의 추천점수
	"activities": string  // 언급된 주요 활동을 리스트형태로 구조화한 형태
}
```


In [29]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

chain = prompt | model | parser

output = chain.invoke({"review": review})

print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'activities': ['해운대 바다 구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4'}
